## Regression Tables via `pf.etable()` and the `Stargazer` package

To produce regression tables, we have two options: pyfixest's internal `etable()` function and the [Stargazer](https://github.com/StatsReporting/stargazer) Python package. The following examples illustrate how to use both options.

Contents:
- [Regression Tables via `pf.etable()`](#regression-tables-via-pfetable)
    - [Basic Usage](#basic-usage)
    - [Keep and drop variables](#keep-and-drop-variables)
    - [Display p-values or confidence intervals](#display-p-values-or-confidence-intervals)
    - [Significance levels and rounding](#significance-levels-and-rounding)
    - [Other output formats](#other-output-formats)
    - [Rename variables](#rename-variables)
    - [Custom model headlines](#custom-model-headlines)
    - [Custom table notes](#custom-table-notes)
    - [Publication-ready LaTex tables](#publication-ready-latex-tables)
- [Regression Tables via `Stargazer`](#regression-tables-via-stargazer)


To begin, we load some libraries and fit a set of regression models. 

In [1]:
import numpy as np
from stargazer.stargazer import LineLocation, Stargazer

import pyfixest as pf

%load_ext autoreload
%autoreload 2

data = pf.get_data()

fit1 = pf.feols("Y ~ X1 + X2 | f1", data = data)
fit2 = pf.feols("Y ~ X1 + X2 | f1 + f2", data = data)
fit3 = pf.feols("Y ~ X1 *X2 | f1 + f2", data = data)
fit4 = pf.feols("Y2 ~ X1 + X2 | f1", data = data)
fit5 = pf.feols("Y2 ~ X1 + X2 | f1 + f2", data = data)
fit6 = pf.feols("Y2 ~ X1 *X2 | f1 + f2", data = data)

## Regression Tables via `pf.etable()`

### Basic Usage

We can compare all regression models via the pyfixest-internal `pf.etable()` function: 

In [2]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6])

### Keep and drop variables
`etable` allows us to do a few things out of the box. For example, we can only keep the variables that we'd like, which keeps all variables that fit the provided regex match. 

In [3]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], keep = "X1")

We can use the `exact_match` argument to select a specific set of variables: 

In [4]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], keep = ["X1", "X2"], exact_match = True)

We can also easily **drop** variables via the `drop` argument: 

In [5]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], drop = ["X1"])

### Display p-values or confidence intervals
By default, `pf.etable()` reports **standard errors**. But we can also ask to output p-values or confidence intervals via the `coef_fmt` function argument. 

In [6]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], coef_fmt = "b \n (se) \n [p]")

### Significance levels and rounding
Additionally, we can also overwrite the defaults for the reported significance levels and control the rounding of results via the `signif_code` and `digits` function arguments: 

In [7]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], signif_code=[0.01, 0.05, 0.1], digits = 5)

### Other output formats
By default, `pf.etable()` returns a data frame, but you can also opt to output markdown or latex via the `type` argument.

In [8]:
# Markdown output:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], signif_code=[0.01, 0.05, 0.1], digits = 5, type = "md")

                      est1          est2          est3          est4          est5          est6
------------  ------------  ------------  ------------  ------------  ------------  ------------
depvar                   Y             Y             Y            Y2            Y2            Y2
------------------------------------------------------------------------------------------------
X1            -0.94953***   -0.92405***   -0.92417***   -1.26655***   -1.23153***   -1.23100***
                 (0.06652)     (0.06093)     (0.06094)     (0.17359)     (0.19228)     (0.19167)
X2            -0.17423***   -0.17411***   -0.18550***   -0.13056***   -0.11767***      -0.07369
                 (0.01840)     (0.01461)     (0.02516)     (0.04239)     (0.04152)     (0.10356)
X1:X2                                         0.01057                                  -0.04082
                                             (0.01818)                                 (0.08093)
---------------------------------

In [9]:
# LaTex output (include latex packages booktabs, threeparttable, and makecell in your document):
tab= pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], signif_code=[0.01, 0.05, 0.1], digits = 2, type = "tex")

The following code for instance generates a pdf including the regression table which you can display clicking on the link below the cell:

In [10]:
import pylatex as pl
from IPython.display import display, FileLink

def make_pdf(tab, file):
    doc = pl.Document()
    doc.packages.append(pl.Package('booktabs'))
    doc.packages.append(pl.Package('threeparttable'))
    doc.packages.append(pl.Package('makecell'))

    with doc.create(pl.Section('A PyFixest LateX Table')):
        with doc.create(pl.Table(position='htbp')) as table:
            table.append(pl.NoEscape(tab))

    doc.generate_pdf(file, clean_tex=False)

make_pdf(tab, "latexdocs/SampleTableDoc")
display(FileLink('latexdocs/SampleTableDoc.pdf'))

c:\Users\dirks\OneDrive\Python\GitHubForks\pyfixest\docs\latexdocs\SampleTableDoc.pdf

### Rename variables
You can also rename variables if you want to have a more readable output. Just pass a dictionary to the `labels` argument. Note that interaction terms will also be relabeled using the specified labels for the interacted variables (if you want to manually relabel an interaction term differently, add it to the dictionary).

In [11]:
labels={
    "Y": "Wage",
    "Y2": "Wealth",
    "X1": "Age",
    "X2": "Years of Schooling",
    "f1": "Industry",
    "f2": "Year"
}

pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], labels=labels)

If you want to label the rows indicating the inclusion of fixed effects not with the variable label but with a custom label, you can pass on a separate dictionary to the `felabels` argument.

In [12]:

pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], labels=labels, felabels={"f1": "Industry Fixed Effects", "f2": "Year Fixed Effects"})

### Custom model headlines
You can also add custom headers for each model by passing a list of strings to the `model_headers` argument.

In [13]:
pf.etable([fit1, fit2, fit3, fit4, fit5, fit6], labels=labels, model_heads=["US", "China", "EU", "US", "China", "EU"])

Or change the ordering of headlines having headlines first and then dependent variables using the `head_order` argument. "hd" stands for headlines then dependent variables, "dh" for dependent variables then headlines. Assigning "d" or "h" can be used to only show dependent variables or only headlines. When head_order="" only model numbers are showm.

In [14]:
pf.etable([fit1, fit4, fit2, fit5, fit3, fit6], labels=labels, 
          model_heads=["US", "US", "China", "China", "EU", "EU"],
          head_order="hd")

Remove the dependent variables from the headers:

In [15]:
pf.etable([fit1, fit4, fit2, fit5, fit3, fit6], labels=labels, 
          model_heads=["US", "US", "China", "China", "EU", "EU"],
          head_order="")

,(1),(2),(3),(4),(5),(6)
Age,-0.950*** (0.067),-1.267*** (0.174),-0.924*** (0.061),-1.232*** (0.192),-0.924*** (0.061),-1.231*** (0.192)
Years of Schooling,-0.174*** (0.018),-0.131** (0.042),-0.174*** (0.015),-0.118** (0.042),-0.185*** (0.025),-0.074 (0.104)
Age × Years of Schooling,,,,,0.011 (0.018),-0.041 (0.081)
Year,-,-,x,x,x,x
Industry,x,x,x,x,x,x
R2,0.489,0.120,0.659,0.172,0.659,0.172
S.E. type,by: f1,by: f1,by: f1,by: f1,by: f1,by: f1
Observations,997,998,997,998,997,998


### Custom table notes
You can replace the default table notes with your own notes using the `notes` argument. 


In [16]:
mynotes="Lorem ipsum dolor sit amet, consetetur sadipscing elitr, sed diam nonumy eirmod tempor invidunt ut labore et dolore magna aliquyam erat, sed diam voluptua. At vero eos et accusam et justo duo dolores et ea rebum. Stet clita kasd gubergren, no sea takimata sanctus est Lorem ipsum dolor sit amet."
pf.etable([fit1, fit4, fit2, fit5, fit3, fit6], labels=labels, 
          model_heads=["US", "US", "China", "China", "EU", "EU"],
          head_order="hd", notes=mynotes)

## Publication-ready LaTex tables 
With few lines of code you thus obtain publication-ready latex tables.

In [17]:
tab=pf.etable([fit1, fit4, fit2, fit5, fit3, fit6], labels=labels, 
          model_heads=["US", "US", "China", "China", "EU", "EU"],
          head_order="hd", type="tex", notes=mynotes)
make_pdf(tab, "latexdocs/SampleTableDoc2")
display(FileLink('latexdocs/SampleTableDoc2.pdf'))

c:\Users\dirks\OneDrive\Python\GitHubForks\pyfixest\docs\latexdocs\SampleTableDoc2.pdf

## Regression Tables via `Stargazer`

We have opened a PR for `pyfixest` support for the excellent [Stargazer](https://github.com/StatsReporting/stargazer/pull/105) project. Until it is merged, you can download the dev version from `py-econometrics` by typing

```bash
pip install git+https://github.com/py-econometrics/stargazer.git
```

`Stargazer` is particularly useful if you need highly customizable regression tables (beyond the scope of `pf.etable()`), or if you want to compare models from `statsmodels` or `linearmodels` with `pyfixest`. 

After installing `stargazer`, we can produce a summary table via the `Stargazer` class: 

In [18]:
stargazer_table = Stargazer([fit1, fit2, fit3, fit4, fit5, fit6])
stargazer_table

We can easily add custom statisics. For example, assume that we want to correct for multiple testing via the Romano-Wolf correction. We can do this as follows:

In [19]:
rwolf_res = pf.rwolf([fit1, fit2, fit3, fit4, fit5, fit6], param = "X1", seed = 123, reps = 9999)
rwolf_pvalues = np.round(rwolf_res.xs("RW Pr(>|t|)"), 3).to_list()

In [20]:
stargazer_table.add_line('Fixed Effects', [x._fixef for x in [fit1, fit2, fit3, fit4, fit5, fit6]], LineLocation.FOOTER_TOP)
stargazer_table.add_line('X1: Romano-Wolf P-Value', rwolf_pvalues, LineLocation.FOOTER_TOP)
stargazer_table